# For both original and revised categories 
control with "revised" flag

In [1]:
!nvidia-smi

Thu May 30 10:45:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:3B:00.0 Off |                    0 |
| N/A   20C    P0              24W / 165W |     18MiB / 24576MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(os.environ["CUDA_VISIBLE_DEVICES"]) 

1


In [2]:
import json
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
revised = True

In [4]:
ROOT = Path('/gaueko0/users/nmishra/multiling_fludetection/data')
translated_dir = ROOT.joinpath('translated/all')

filename = 'alldata_revisedcateg' if revised else 'alldata'
print(filename)

alldata = pd.read_csv(ROOT.joinpath(f'all/{filename}.csv'))
alldata = alldata.convert_dtypes()
print(alldata.shape)

alldata_revisedcateg
(4284, 6)


In [5]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                4284 non-null   Int64 
 1   lang              4284 non-null   string
 2   year              4284 non-null   Int64 
 3   tweet             4283 non-null   string
 4   final_annotation  4284 non-null   string
 5   overlap           4284 non-null   Int64 
dtypes: Int64(3), string(3)
memory usage: 213.5 KB


In [6]:
translated_files = [i for i in translated_dir.glob('*.jsonl') if 'translatedto' in i.stem]
all_translated = []
for file in translated_files:
    df = pd.read_json(file, lines=True, encoding='utf-8', dtype={'id': 'int64'})
    df['target_lang'] = file.stem[-2:]
    all_translated.append(df)
all_translated = pd.concat(all_translated)
print(all_translated.shape[0]/4)
all_translated.info()
# all_translated.sort_values('id').to_csv(translated_dir.joinpath('translation.csv'), index=False)

4400.0
<class 'pandas.core.frame.DataFrame'>
Index: 17600 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                17600 non-null  int64 
 1   translated_tweet  17600 non-null  object
 2   target_lang       17600 non-null  object
dtypes: int64(1), object(2)
memory usage: 550.0+ KB


In [7]:
# combine remaining attributes of the associated id
all_translated_merged = all_translated.merge(alldata.drop('tweet',axis=1), on='id', how='left')
all_translated_merged.rename(columns={'translated_tweet':'tweet'}, inplace=True)
all_translated_merged = all_translated_merged.convert_dtypes()
all_translated_merged = all_translated_merged[all_translated_merged['id'].isin(alldata['id'])]
print(all_translated_merged.shape)
all_translated_merged.head()
# all_translated_merged['final_annotation'].value_counts().sum()

(17136, 7)


,id,tweet,target_lang,lang,year,final_annotation,overlap
0,1068447281270136832,@user Yrr bs flu fever chal rha continuously 😪😫,es,en,2018,1. Likely ILI infection,1
1,1045811659229605888,"(2/2) ...En segundo lugar, dado que los virus ...",es,en,2018,3. Not Related to ILI or COVID-19 Infection,1
2,1052804290509602816,@user Yo he tenido gripe de verdad una vez. Ac...,es,en,2018,3. Not Related to ILI or COVID-19 Infection,1
3,1078661659181355008,Esta es la primera vez en mi vida que tengo la...,es,en,2018,1. Likely ILI infection,1
4,1047584640922525696,Esta gripe me tiene débil af URL,es,en,2018,1. Likely ILI infection,1


In [8]:
all_translated_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17136 entries, 0 to 17599
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                17136 non-null  Int64 
 1   tweet             17136 non-null  string
 2   target_lang       17136 non-null  string
 3   lang              17136 non-null  string
 4   year              17136 non-null  Int64 
 5   final_annotation  17136 non-null  string
 6   overlap           17136 non-null  Int64 
dtypes: Int64(3), string(4)
memory usage: 1.1 MB


In [9]:
# add original data
alldata['target_lang'] = alldata['lang']
full_data = pd.concat([all_translated_merged, alldata])
print(full_data.shape)
full_data[full_data['id']==1068447281270136832]

(21420, 7)


,id,tweet,target_lang,lang,year,final_annotation,overlap
0,1068447281270136832,@user Yrr bs flu fever chal rha continuously 😪😫,es,en,2018,1. Likely ILI infection,1
1200,1068447281270136832,@user Yrr bs influenza febbre chal rha continu...,it,en,2018,1. Likely ILI infection,1
5600,1068447281270136832,@user Yrr bs flu fever chal rha continuously 😪😫😫,fr,en,2018,1. Likely ILI infection,1
14400,1068447281270136832,@user Yrr bs Grippefieber chal rha kontinuierl...,de,en,2018,1. Likely ILI infection,1
382,1068447281270136832,@user Yrr bs flu fever chal rha continuously 😪😫,en,en,2018,1. Likely ILI infection,1


In [10]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21420 entries, 0 to 4283
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                21420 non-null  Int64 
 1   tweet             21419 non-null  string
 2   target_lang       21420 non-null  string
 3   lang              21420 non-null  string
 4   year              21420 non-null  Int64 
 5   final_annotation  21420 non-null  string
 6   overlap           21420 non-null  Int64 
dtypes: Int64(3), string(4)
memory usage: 1.4 MB


In [11]:
full_data[full_data['final_annotation'].isna()]

,id,tweet,target_lang,lang,year,final_annotation,overlap


In [12]:
full_data.shape

(21420, 7)

In [13]:
alldata[alldata['id']==1107333327948636160]

,id,lang,year,tweet,final_annotation,overlap,target_lang


In [14]:
full_data['eval_lang'] =  full_data['lang'] + '-' + full_data['target_lang']
full_data.groupby('eval_lang').count()

,id,tweet,target_lang,lang,year,final_annotation,overlap
eval_lang,,,,,,,
de-de,967,967,967,967,967,967,967
de-en,967,967,967,967,967,967,967
de-es,967,967,967,967,967,967,967
de-fr,967,967,967,967,967,967,967
de-it,967,967,967,967,967,967,967
en-de,196,196,196,196,196,196,196
en-en,196,196,196,196,196,196,196
en-es,196,196,196,196,196,196,196
en-fr,196,196,196,196,196,196,196


In [17]:
full_data['original'] = full_data['lang']==full_data['target_lang']
full_data

,id,tweet,target_lang,lang,year,final_annotation,overlap,eval_lang,original
0,1068447281270136832,@user Yrr bs flu fever chal rha continuously 😪😫,es,en,2018,1. Likely ILI infection,1,en-es,False
1,1045811659229605888,"(2/2) ...En segundo lugar, dado que los virus ...",es,en,2018,3. Not Related to ILI or COVID-19 Infection,1,en-es,False
2,1052804290509602816,@user Yo he tenido gripe de verdad una vez. Ac...,es,en,2018,3. Not Related to ILI or COVID-19 Infection,1,en-es,False
3,1078661659181355008,Esta es la primera vez en mi vida que tengo la...,es,en,2018,1. Likely ILI infection,1,en-es,False
4,1047584640922525696,Esta gripe me tiene débil af URL,es,en,2018,1. Likely ILI infection,1,en-es,False
...,...,...,...,...,...,...,...,...,...
4279,1635992287149686784,Mi conforta sapere che il grande Prof #Parsi m...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True
4280,1631445097094029312,@user @user Mascherina che visti i morti che l...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True
4281,1630308873075671041,@user Io tre giorni a letto con l’influenza e ...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True
4282,1633751928957022209,@user Guardano a una confederazione puramente ...,it,it,2023,3. Not Related to ILI or COVID-19 Infection,0,it-it,True


In [18]:
out_filename = 'fulldata_revisedcateg' if revised else 'fulldata'
full_data.sort_values('id').to_csv(ROOT.joinpath(f'all/{out_filename}.csv'), index=False)